<a href="https://colab.research.google.com/github/deterministic-algorithms-lab/wikipron-modeling/blob/master/wikipron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wikipron

In [ ]:
!pip install pynini ngram openfst

In [ ]:
!git clone https://github.com/pytorch/fairseq
%cd fairseq
!pip install --editable ./

In [ ]:
%cd ..

In [ ]:
lg = 'hin'

In [ ]:
!git clone https://github.com/deterministic-algorithms-lab/wikipron-modeling

In [ ]:
%cd wikipron-modeling/
!mkdir data

In [ ]:
%cd data
!mkdir {train,test,dev}

In [ ]:
#!wikipron {lg} > {lg}.tsv
!shuf {lg}.tsv > shuf_{lg}.tsv
!rm {lg}.tsv
!mv shuf_{lg}.tsv {lg}.tsv

In [ ]:
#Cleans Data & splits it(80:10:10). [Remove short words + NFC normalisation + resolve multiple gold targets for same word]
!bash ../scripts/prepare

In [ ]:
!mv {lg}_dev.tsv ./dev/
!mv {lg}_train.tsv ./train/
!mv {lg}_test.tsv ./test/

In [ ]:
%cd ../nn

In [ ]:
!bash ./preprocess

In [ ]:
!fairseq-preprocess --source-lang {lg}.graphemes --target-lang {lg}.phonemes --trainpref ./data-txt/train \
--testpref ./data-txt/test --validpref ./data-txt/dev --destdir ./data-bin/{lg}/

In [ ]:
!bash ./sweep